In [6]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Activation, LeakyReLU, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import zipfile

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("xhlulu/140k-real-and-fake-faces")

print("Path to dataset files:", path)

100%|██████████| 3.75G/3.75G [00:39<00:00, 101MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2


In [7]:
# Path to dataset files
dataset_path = '/root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2'


In [10]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2"

# List files and directories in the dataset path
print(os.listdir(dataset_path))


['real_vs_fake', 'train.csv', 'test.csv', 'valid.csv']


In [13]:
import os
import pandas as pd

dataset_path = "/root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2"

# Check contents of real_vs_fake folder
real_fake_path = os.path.join(dataset_path, 'real_vs_fake')
print("Contents of 'real_vs_fake':", os.listdir(real_fake_path))

# Check the first few rows of CSV files
train_csv_path = os.path.join(dataset_path, 'train.csv')
test_csv_path = os.path.join(dataset_path, 'test.csv')
valid_csv_path = os.path.join(dataset_path, 'valid.csv')

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)
valid_df = pd.read_csv(valid_csv_path)

print("Train CSV Sample:\n", train_df.head())
print("Test CSV Sample:\n", test_df.head())
print("Valid CSV Sample:\n", valid_df.head())


Contents of 'real_vs_fake': ['real-vs-fake']
Train CSV Sample:
    Unnamed: 0                                      original_path     id  \
0           0  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  31355   
1           1  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  02884   
2           2  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  33988   
3           3  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  53875   
4           4  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  24149   

   label label_str                  path  
0      1      real  train/real/31355.jpg  
1      1      real  train/real/02884.jpg  
2      1      real  train/real/33988.jpg  
3      1      real  train/real/53875.jpg  
4      1      real  train/real/24149.jpg  
Test CSV Sample:
    Unnamed: 0                                      original_path     id  \
0           0  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  18233   
1           1  /kaggle/input/flickrfaceshq-dataset-nvidia-p

In [15]:
import os
dataset_path = '/root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2'
print(os.listdir(dataset_path))
print(os.listdir(os.path.join(dataset_path, 'real_vs_fake')))


['real_vs_fake', 'train.csv', 'test.csv', 'valid.csv']
['real-vs-fake']


In [16]:
import pandas as pd
df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))
print(df.head())  # Ensure 'path' column exists and contains valid image paths


   Unnamed: 0                                      original_path     id  \
0           0  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  31355   
1           1  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  02884   
2           2  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  33988   
3           3  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  53875   
4           4  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  24149   

   label label_str                  path  
0      1      real  train/real/31355.jpg  
1      1      real  train/real/02884.jpg  
2      1      real  train/real/33988.jpg  
3      1      real  train/real/53875.jpg  
4      1      real  train/real/24149.jpg  


In [23]:
# Adjusted Preprocessing Function
def preprocess_images(csv_file):
    df = pd.read_csv(csv_file)
    images = []
    for img_path in df['path']:
        full_path = os.path.join(dataset_path, 'real_vs_fake', img_path.replace("\\", "/"))  # Ensure correct path format
        if os.path.exists(full_path):
            img = load_img(full_path, target_size=(128, 128))
            img = img_to_array(img) / 255.0
            images.append(img)
        else:
            print(f"Missing file: {full_path}")  # Debugging missing files
    return np.array(images)


In [24]:
# Load and preprocess images
train_images = preprocess_images(os.path.join(dataset_path, 'train.csv'))
valid_images = preprocess_images(os.path.join(dataset_path, 'valid.csv'))
test_images = preprocess_images(os.path.join(dataset_path, 'test.csv'))


Streaming output truncated to the last 5000 lines.
Missing file: /root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2/real_vs_fake/train/fake/5INLD8DESJ.jpg
Missing file: /root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2/real_vs_fake/train/fake/SRV5I7HTPB.jpg
Missing file: /root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2/real_vs_fake/train/fake/21CDSEAZX8.jpg
Missing file: /root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2/real_vs_fake/train/fake/EOE9LO2E58.jpg
Missing file: /root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2/real_vs_fake/train/fake/67V6Z2J3V1.jpg
Missing file: /root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2/real_vs_fake/train/fake/07V6YECRK3.jpg
Missing file: /root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2/real_vs_fake/train/fake/BT9PRWBGRX.jpg
Missing file: /root/.cache/kagglehub/datasets/xhlulu/

KeyboardInterrupt: 

In [18]:
# Define Conditional GAN Generator
def build_generator():
    model = Sequential()
    model.add(Dense(128 * 16 * 16, activation="relu", input_dim=100))
    model.add(Reshape((16, 16, 128)))
    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2DTranspose(3, kernel_size=4, strides=2, padding="same", activation="tanh"))
    model.add(Reshape(IMG_SHAPE))
    return model

def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=4, strides=2, input_shape=IMG_SHAPE, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

In [19]:
# Compile Models
generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

discriminator.trainable = False
z = Input(shape=(100,))
generated_image = generator(z)
validity = discriminator(generated_image)
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [20]:
# Training Parameters
EPOCHS = 100
BATCH_SIZE = 64
SAVE_INTERVAL = 500

In [22]:
# Training Loop
def train():
    for epoch in range(EPOCHS):
        idx = np.random.randint(0, train_images.shape[0], BATCH_SIZE)
        real_images = train_images[idx]
        noise = np.random.normal(0, 1, (BATCH_SIZE, 100))
        fake_images = generator.predict(noise)

        real_labels = np.ones((BATCH_SIZE, 1))
        fake_labels = np.zeros((BATCH_SIZE, 1))

        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        g_loss = combined.train_on_batch(noise, real_labels)

        if epoch % SAVE_INTERVAL == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
